In [1]:
!pip install --upgrade numpy

In [1]:
import numpy as np
import os
import statistics 
from tqdm.notebook import tqdm
import torch
from torch.optim import Adam
import matplotlib.pyplot as plt

In [2]:
# Usamos cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
l = os.listdir('Train np')
lis=[]
for arch in l:
    if arch.startswith('x_'):
        lis.append(arch)

In [4]:
mean = np.load('Res/x_mean.npy')
std = np.load('Res/x_std.npy')

In [5]:
X_train=[]
Y_train=[]
for mus in tqdm(lis):
    x = np.load('Train np/'+mus)
    y = np.load('Train np/'+mus.replace('x_','y_'))
    x = np.nan_to_num((x - mean)/std)
    x = np.pad(x, ((3,3), (0,0)), 'constant', constant_values = 0)
    for i in range(x.shape[0]-6):
        X_train.append(torch.from_numpy(x[i:i+7]).float())
        Y_train.append(torch.from_numpy(y[i]).float())

  0%|          | 0/210 [00:00<?, ?it/s]

In [6]:
l = os.listdir('Test np')
lis=[]
for arch in l:
    if arch.startswith('x_'):
        lis.append(arch)

In [7]:
X_test=[]
Y_test=[]
for mus in tqdm(lis):
    x = np.load('Test np/'+mus)
    y = np.load('Test np/'+mus.replace('x_','y_'))
    x = np.nan_to_num((x - mean)/std)
    x = np.pad(x, ((3,3), (0,0)), 'constant', constant_values = 0)
    for i in range(x.shape[0]-6):
        X_test.append(torch.from_numpy(x[i:i+7]).float())
        Y_test.append(torch.from_numpy(y[i]).float())

  0%|          | 0/60 [00:00<?, ?it/s]

In [8]:
from torch.utils.data import Dataset

class TrainData(Dataset):
    def __init__(self):
        self.samples = X_train
        self.labels = Y_train

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

In [9]:
class TestData(Dataset):
    def __init__(self):
        self.samples = X_test
        self.labels = Y_test

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

In [10]:
Traindata = TrainData()
Testdata = TestData()

In [11]:
batch_size = 500
train_loader = torch.utils.data.DataLoader(dataset=Traindata, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=Testdata, batch_size=batch_size, shuffle=True)

In [12]:
import torch.nn as nn

In [21]:
def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    Tl=[]
    Vl=[]
    Va=[]
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            X = X.to(device)
            Y = Y.to(device)
            Y_preds = model(X)
            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(i)
        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        Tl.append(torch.tensor(losses).mean())
        vl = CalcValLossAndAccuracy(model, loss_fn, val_loader)
        Vl.append(vl)
    plt.plot(Tl, label = "Loss Entrenamiento")
    plt.plot(Vl, label= "Loss Validacion")
    plt.legend()
    plt.show()

In [20]:
#Calcular perdida y accuracy de test
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in tqdm(val_loader):
            X = X.to(device)
            Y = Y.to(device)
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))
        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        return(torch.tensor(losses).mean())

In [17]:
class create_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(5,25), padding=0)
        self.pool = nn.MaxPool2d(kernel_size=(1,3), stride=1)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=(3,5), padding=0)
        self.linear1 = nn.Linear(11000, 1000)
        self.linear2 = nn.Linear(1000, 200)
        self.linear3 = nn.Linear(200, 88)
        self.Drop   = nn.Dropout(0.2)
        self.ReLU   = nn.ReLU()
        self.Sig     = nn.Sigmoid()
        
        
    def forward(self, x): 
        x = x.reshape(-1, 7,252, 1)
        x = x.permute(0, 3, 1, 2)
        x = self.Drop(self.ReLU(self.conv1(x)))
        x = self.pool(x)
        x = self.Drop(self.ReLU(self.conv2(x)))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.Drop(self.ReLU(self.linear1(x)))
        x = self.Drop(self.ReLU(self.linear2(x)))
        x = self.Sig(self.linear3(x))
        return x

In [22]:
epochs = 5
learning_rate = 1e-4

loss_fn = nn.BCELoss()
classifier = create_model()
classifier.to(device)
optimizer = Adam(classifier.parameters(), lr=learning_rate)

TrainModel(classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

  0%|          | 0/8498 [00:00<?, ?it/s]

1
Train Loss : 0.058


  0%|          | 0/2699 [00:00<?, ?it/s]

Valid Loss : 0.105


  0%|          | 0/8498 [00:00<?, ?it/s]

...

  0%|          | 0/8498 [00:00<?, ?it/s]

5
Train Loss : 0.033


  0%|          | 0/2699 [00:00<?, ?it/s]

Valid Loss : 0.125


NameError: name 'plt' is not defined

In [40]:
torch.save(classifier.state_dict(), 'Res/model')

In [32]:
classifier.to('cpu')
classifier.eval()

create_model(
  (conv1): Conv2d(1, 50, kernel_size=(5, 25), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(1, 3), stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 50, kernel_size=(3, 5), stride=(1, 1))
  (linear1): Linear(in_features=11000, out_features=1000, bias=True)
  (linear2): Linear(in_features=1000, out_features=200, bias=True)
  (linear3): Linear(in_features=200, out_features=88, bias=True)
  (Drop): Dropout(p=0.2, inplace=False)
  (ReLU): ReLU()
  (Sig): Sigmoid()
)

In [28]:
X=[]
X_test = x = np.load('Test np/x_MAPS_MUS-mz_331_3_ENSTDkCl.npy')
X_test = np.nan_to_num((X_test - mean)/std)
X_test = np.pad(X_test, ((3,3), (0,0)), 'constant', constant_values = 0)
for i in range(x.shape[0]-6):
    X.append(torch.from_numpy(X_test[i:i+7]).float())

In [33]:
Y = []
for k in X:
    Y.append(classifier(k))

In [34]:
y_hat = torch.cat(Y)

In [37]:
y_hat = y_hat.detach().numpy()

In [38]:
Y_pred = 1.0 * ( np.array(y_hat) > 0.5)

In [39]:
np.save('Res/y_hat.npy', Y_pred)